# Content Repository Update (Version 2)
**This notebook works in BOTH Databricks and local Jupyter**

**PREREQUISITE: Run 1_content_repo_ingestion.ipynb first to create tables and ingest v1!**

This notebook demonstrates updating content in the repository with version 2 files:

1. **Ingest Updated Content**: Add version 2 of edited documents to `content_repo`

**Note**: This notebook ONLY handles content repository ingestion. Chunking and change detection is handled in 4_change_detection.ipynb.

## Purpose
- Version 1 files were ingested in `1_content_repo_ingestion.ipynb`
- Content has been edited (policy changes, updates, corrections)
- Version 2 files need to be ingested into `content_repo`
- Same file names, different version numbers (`raw_file_version_nbr = 2`)

## Workflow
```
v1 Content (done in 1_content_repo_ingestion.ipynb)
  ↓
v2 Content (this notebook)
  ↓
4_change_detection.ipynb (chunk + detect changes)
```

In [9]:
# Install required packages
%pip install loguru pandas -q

Note: you may need to restart the kernel to use updated packages.


In [10]:
# Restart Python (Databricks only)
try:
    dbutils.library.restartPython()
except NameError:
    print("✓ Local environment - no restart needed")

✓ Local environment - no restart needed


In [11]:
# CONFIGURATION - Using Repository Pattern (Phase 5)

from utility.environment import is_databricks
from database.config import DatabaseConfig

if is_databricks():
    # Databricks: Setup widgets
    dbutils.widgets.text("catalog", "onedata_us_east_1_shared_dit", "Catalog")
    dbutils.widgets.text("schema", "ssot_raw_ditagen_dit", "Schema")
    dbutils.widgets.text("volume", "datacloud-dita-gen-volume", "Volume")
    
    CATALOG = dbutils.widgets.get("catalog")
    SCHEMA = dbutils.widgets.get("schema")
    VOLUME = dbutils.widgets.get("volume")
    
    # Databricks paths
    DATA_PATH = f"/Volumes/{CATALOG}/{SCHEMA}/{VOLUME}/FAQ_update/data_ingestion/data/5_sample_content_repo_v2_edited.csv"
    
    # Create database configuration
    config = DatabaseConfig(backend='databricks', catalog=CATALOG, schema=SCHEMA)
    
    print(f"📊 Databricks mode: {CATALOG}.{SCHEMA}")
    print(f"📂 Data path: {DATA_PATH}")
else:
    # Local environment
    DATA_PATH = "data_ingestion/data/5_sample_content_repo_v2_edited.csv"  # Relative path
    DB_PATH = "databases/faq_update.db"  # Must match 0_database_setup.ipynb
    
    # Create database configuration
    config = DatabaseConfig(backend='sqlite', db_path=DB_PATH)
    
    print(f"💾 Local mode")
    print(f"🗄️  Database: {DB_PATH}")
    print(f"📂 Data path: {DATA_PATH}")

print()
print(f"⚙️  Backend: {config.backend}")
print()

💾 Local mode
🗄️  Database: databases/faq_update.db
📂 Data path: data_ingestion/data/5_sample_content_repo_v2_edited.csv

⚙️  Backend: sqlite



In [12]:
# CHECK: Verify backend connection and tables using Repository

from database.backends.factory import BackendFactory
from database.repository import ContentRepository

print("=" * 70)
print("PREREQUISITE CHECK - Repository Pattern")
print("=" * 70)
print()

# Create backend and repository
backend = BackendFactory.create_backend(config)
content_repo = ContentRepository(backend)

print(f"✓ Backend created: {backend.__class__.__name__}")
print(f"✓ Repository created: ContentRepository")
print()

# Check that we can query the database
try:
    stats = content_repo.get_file_stats()
    print(f"✓ Database connection successful")
    print(f"✓ Current file count: {stats.get('total_files', 0)}")
    print()
except Exception as e:
    print(f"❌ ERROR: Could not connect to database")
    print(f"   {str(e)}")
    print()
    print("⚠️  Please run 0_database_setup.ipynb first to create all tables.")
    raise

print("=" * 70)
print("✅ PREREQUISITE CHECK PASSED")
print("=" * 70)
print()

2025-11-02 14:42:23.490 | DEBUG    | database.backends.factory:create_backend:116 - Returning existing backend singleton
2025-11-02 14:42:23.505 | DEBUG    | database.repository.content_repository:get_file_stats:837 - Calculating file statistics
2025-11-02 14:42:23.510 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 1 rows
2025-11-02 14:42:23.511 | DEBUG    | database.repository.base:execute_query:590 - Query returned 1 rows
2025-11-02 14:42:23.512 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 1 rows
2025-11-02 14:42:23.513 | DEBUG    | database.repository.base:execute_query:590 - Query returned 1 rows
2025-11-02 14:42:23.514 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 1 rows
2025-11-02 14:42:23.516 | DEBUG    | database.repository.base:execute_query:590 - Query returned 1 rows
2025-11-02 14:42:23.518 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 1

PREREQUISITE CHECK - Repository Pattern

✓ Backend created: SQLiteBackend
✓ Repository created: ContentRepository

✓ Database connection successful
✓ Current file count: 14

✅ PREREQUISITE CHECK PASSED



In [13]:
# INGEST VERSION 2 CONTENT REPOSITORY - Using ContentRepository

import pandas as pd

print("=" * 70)
print("VERSION 2 CONTENT REPOSITORY INGESTION - Repository Pattern")
print("=" * 70)
print()

# Load data
print(f"📂 Loading data from: {DATA_PATH}")

if is_databricks():
    # Databricks: Use spark to read CSV
    df = spark.read.csv(DATA_PATH, header=True, inferSchema=True).toPandas()
else:
    # Local: Use pandas
    df = pd.read_csv(DATA_PATH)

df = df.drop(columns=['content_checksum'], errors='ignore')

print(f"✓ Loaded {len(df)} records")
print()
print("📋 Sample data:")
print(df.head(3))
print()

# Get stats before
stats_before = content_repo.get_file_stats()
print("📊 Before ingestion:")
print(f"   Total files: {stats_before.get('total_files', 0)}")
print(f"   Unique files: {stats_before.get('unique_files', 0)}")
print(f"   File types: {stats_before.get('by_type', {})}")
print(f"   File statuses: {stats_before.get('by_status', {})}")
print()

# Ingest using repository directly
print("💾 Ingesting version 2 data using ContentRepository...")
try:
    result = content_repo.ingest_files_bulk(
        files_df=df,
        validate=True,
        if_exists='append'
    )
    
    print(f"✅ SUCCESS")
    print(f"   Rows inserted: {result.get('rows_inserted', 0)}")
    print(f"   Message: {result.get('message', 'Ingestion completed')}")
except Exception as e:
    print(f"❌ FAILED")
    print(f"   Error: {str(e)}")
    raise

print()

# Get stats after
stats_after = content_repo.get_file_stats()
print("📊 After ingestion:")
print(f"   Total files: {stats_after.get('total_files', 0)}")
print(f"   Unique files: {stats_after.get('unique_files', 0)}")
print(f"   File types: {stats_after.get('by_type', {})}")
print(f"   File statuses: {stats_after.get('by_status', {})}")
print()

print("=" * 70)
print("✅ INGESTION COMPLETE")
print("=" * 70)

VERSION 2 CONTENT REPOSITORY INGESTION - Repository Pattern

📂 Loading data from: data_ingestion/data/5_sample_content_repo_v2_edited.csv
✓ Loaded 10 records

2025-11-02 14:42:23.687 | DEBUG    | database.repository.content_repository:get_file_stats:837 - Calculating file statistics




📋 Sample data:
             raw_file_nme raw_file_type  raw_file_version_nbr  \
0   Employee_Handbook.pdf           pdf                     2   
1  IT_Security_Policy.pdf           pdf                     2   
2  Remote_Work_Policy.pdf           pdf                     2   

                                    raw_file_path  \
0   data_ingestion/data/raw/Employee_Handbook.pdf   
1  data_ingestion/data/raw/IT_Security_Policy.pdf   
2  data_ingestion/data/raw/Remote_Work_Policy.pdf   

                        extracted_markdown_file_path  \
0  data_ingestion/data/markdown/Employee_Handbook...   
1  data_ingestion/data/markdown/IT_Security_Polic...   
2  data_ingestion/data/markdown/Remote_Work_Polic...   

                                           title_nme file_status  
0                   Employee Handbook 2025 - Updated      Active  
1  IT Security Policy - Enhanced Password Require...      Active  
2  Remote Work Policy - Updated Guidelines (3 day...      Active  



2025-11-02 14:42:23.687 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 1 rows
2025-11-02 14:42:23.687 | DEBUG    | database.repository.base:execute_query:590 - Query returned 1 rows
2025-11-02 14:42:23.687 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 1 rows
2025-11-02 14:42:23.687 | DEBUG    | database.repository.base:execute_query:590 - Query returned 1 rows
2025-11-02 14:42:23.687 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 1 rows
2025-11-02 14:42:23.687 | DEBUG    | database.repository.base:execute_query:590 - Query returned 1 rows
2025-11-02 14:42:23.687 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 1 rows
2025-11-02 14:42:23.687 | DEBUG    | database.repository.base:execute_query:590 - Query returned 1 rows
2025-11-02 14:42:23.687 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 1 rows
2025-11-02 14:42:23.687 

📊 Before ingestion:
   Total files: 14
   Unique files: 14
   File types: {'pdf': 14}
   File statuses: {'Active': 14}

💾 Ingesting version 2 data using ContentRepository...
✅ SUCCESS
   Rows inserted: 10
   Message: Successfully inserted 10 rows into content_repo

📊 After ingestion:
   Total files: 24
   Unique files: 16
   File types: {'pdf': 24}
   File statuses: {'Active': 24}

✅ INGESTION COMPLETE


In [14]:
# (OPTIONAL) Verify the ingestion using Repository

print()
print("=" * 70)
print("VERIFICATION - Repository Pattern")
print("=" * 70)
print()

# Use repository to list all files
all_files = content_repo.list_files(order_by="raw_file_version_nbr, raw_file_nme")

print(f"📋 All files in repository ({len(all_files)} total):")
print()

# Group by file name to show versions
from collections import defaultdict
files_by_name = defaultdict(list)
for file in all_files:
    files_by_name[file['raw_file_nme']].append(file)

# Display grouped
for filename in sorted(files_by_name.keys()):
    versions = files_by_name[filename]
    print(f"📄 {filename}:")
    for v in sorted(versions, key=lambda x: x['raw_file_version_nbr']):
        print(f"   v{v['raw_file_version_nbr']}: {v['title_nme']} (id={v['ud_source_file_id']}, status={v['file_status']})")
    print()

print("=" * 70)
print("✅ VERIFICATION COMPLETE")
print("=" * 70)

2025-11-02 14:42:23.823 | DEBUG    | database.repository.content_repository:list_files:345 - Listing files (status=None, type=None, limit=None, offset=None)
2025-11-02 14:42:23.831 | DEBUG    | database.backends.sqlite.backend:execute_query:239 - Query returned 24 rows
2025-11-02 14:42:23.831 | DEBUG    | database.repository.base:execute_query:590 - Query returned 24 rows
2025-11-02 14:42:23.831 | DEBUG    | database.repository.content_repository:list_files:382 - Found 24 files



VERIFICATION - Repository Pattern

📋 All files in repository (24 total):

📄 Benefits_Guide.pdf:
   v1: Employee Benefits Overview - Health Insurance (id=4, status=Active)

📄 Code_of_Conduct.pdf:
   v1: Code of Conduct - Ethics and Compliance (id=6, status=Active)

📄 Email_Policy.pdf:
   v1: Email Usage Policy (id=13, status=Active)
   v2: Email Usage Policy - Updated Wording (id=23, status=Active)

📄 Employee_Handbook.pdf:
   v1: Employee Handbook 2024 (id=1, status=Active)
   v2: Employee Handbook 2025 - Updated (id=15, status=Active)

📄 Equipment_Return.pdf:
   v1: Equipment Return Policy - New (id=24, status=Active)

📄 Expense_Reimbursement.pdf:
   v1: Expense Reimbursement Policy - Submission Guidelines (id=8, status=Active)

📄 Health_Insurance.pdf:
   v1: Health Insurance Enrollment - Three Plans (id=11, status=Active)
   v2: Health Insurance Enrollment - Four Plans (id=21, status=Active)

📄 IT_Security_Policy.pdf:
   v1: IT Security Policy - Password Requirements (id=2, status=A

In [15]:
stats_after

{'total_files': 24,
 'by_status': {'Active': 24},
 'by_type': {'pdf': 24},
 'total_versions': 32,
 'unique_files': 16,
 'latest_file': {'name': 'Employee_Handbook.pdf',
  'created_at': '2025-11-02 14:42:23.718163'}}

In [16]:
# Import the database module
from database import get_backend

# Get a backend instance (auto-detects SQLite or Databricks based on environment)
backend = get_backend()

# Get dataframe from content_repo table (if it exists)
df_content_repo = backend.read_table("content_repo")

# # Alternative: Get dataframe from content_checksums table (the newer table schema)
# df_content_checksums = backend.read_table("content_checksums")

# Display the results
print("Content Repo DataFrame:")
df_content_repo
# or
# print("Content Checksums DataFrame:")
# print(df_content_checksums)

2025-11-02 14:42:23.922 | DEBUG    | database.backends.factory:create_backend:116 - Returning existing backend singleton
2025-11-02 14:42:23.928 | DEBUG    | database.backends.sqlite.backend:read_table:486 - Read 24 rows from content_repo


Content Repo DataFrame:


ud_source_file_id               raw_file_nme raw_file_type  \
0                   1      Employee_Handbook.pdf           pdf   
1                   2     IT_Security_Policy.pdf           pdf   
2                   3     Remote_Work_Policy.pdf           pdf   
3                   4         Benefits_Guide.pdf           pdf   
4                   5     Payroll_Processing.pdf           pdf   
5                   6        Code_of_Conduct.pdf           pdf   
6                   7          Travel_Policy.pdf           pdf   
7                   8  Expense_Reimbursement.pdf           pdf   
8                   9    Performance_Reviews.pdf           pdf   
9                  10         Parental_Leave.pdf           pdf   
10                 11       Health_Insurance.pdf           pdf   
11                 12        Vacation_Policy.pdf           pdf   
12                 13           Email_Policy.pdf           pdf   
13                 14         Parking_Policy.pdf           pdf   
14                 15      Employee_Handbook.pdf           pdf   
15                 16     IT_Security_Policy.pdf           pdf   
16                 17     Remote_Work_Policy.pdf           pdf   
17                 18          Travel_Policy.pdf           pdf   
18                 19         Parental_Leave.pdf           pdf   
19                 20       Workplace_Safety.pdf           pdf   
20                 21       Health_Insurance.pdf           pdf   
21                 22        Vacation_Policy.pdf           pdf   
22                 23           Email_Policy.pdf           pdf   
23                 24       Equipment_Return.pdf           pdf   

    raw_file_version_nbr                                      raw_file_path  \
0                      1      data_ingestion/data/raw/Employee_Handbook.pdf   
1                      1     data_ingestion/data/raw/IT_Security_Policy.pdf   
2                      1     data_ingestion/data/raw/Remote_Work_Policy.pdf   
3                      1         data_ingestion/data/raw/Benefits_Guide.pdf   
4                      1     data_ingestion/data/raw/Payroll_Processing.pdf   
5                      1        data_ingestion/data/raw/Code_of_Conduct.pdf   
6                      1          data_ingestion/data/raw/Travel_Policy.pdf   
7                      1  data_ingestion/data/raw/Expense_Reimbursement.pdf   
8                      1    data_ingestion/data/raw/Performance_Reviews.pdf   
9                      1         data_ingestion/data/raw/Parental_Leave.pdf   
10                     1       data_ingestion/data/raw/Health_Insurance.pdf   
11                     1        data_ingestion/data/raw/Vacation_Policy.pdf   
12                     1           data_ingestion/data/raw/Email_Policy.pdf   
13                     1         data_ingestion/data/raw/Parking_Policy.pdf   
14                     2      data_ingestion/data/raw/Employee_Handbook.pdf   
15                     2     data_ingestion/data/raw/IT_Security_Policy.pdf   
16                     2     data_ingestion/data/raw/Remote_Work_Policy.pdf   
17                     2          data_ingestion/data/raw/Travel_Policy.pdf   
18                     2         data_ingestion/data/raw/Parental_Leave.pdf   
19                     1       data_ingestion/data/raw/Workplace_Safety.pdf   
20                     2       data_ingestion/data/raw/Health_Insurance.pdf   
21                     2        data_ingestion/data/raw/Vacation_Policy.pdf   
22                     2           data_ingestion/data/raw/Email_Policy.pdf   
23                     1       data_ingestion/data/raw/Equipment_Return.pdf   

                         extracted_markdown_file_path  \
0   data_ingestion/data/markdown/Employee_Handbook...   
1   data_ingestion/data/markdown/IT_Security_Polic...   
2   data_ingestion/data/markdown/Remote_Work_Polic...   
3   data_ingestion/data/markdown/Benefits_Guide_v1.md   
4   data_ingestion/data/markdown/Payroll_Processin...   
5   data_ingestion/data/markdown/Co